In [1]:
import requests
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from typhoon_ocr import ocr_document

In [2]:
def scrape_event_links(url, suffix):
    """
    Fetches the MyTCAS page, finds all <a> tags within the
    't-sec-events' container, and filters them based on href suffixes.
    """
    try:
        # 1. Fetch the HTML content
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        response.encoding = 'utf-8'  # Handle Thai characters

        # 2. Parse the HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # 3. Find the main container div
        main_container = soup.find('div', class_='s-container t-sec-events')
        
        if not main_container:
            print("Error: Could not find the main event container (div.s-container.t-sec-events).")
            return None

        # 4. Find all <a> tags and filter them
        for a_tag in main_container.find_all('a'):
            href = a_tag.get('href')
            
            # Check if href exists and ends with one of the target suffixes
            if href and href.endswith(suffix):
                return href
                

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        return None

In [3]:
def ocr_image_from_url(image_url: str, output_md_filename: str):
    
    # Save the file in the same directory as the script
    local_filename = "temp_ocr_image.jpg"
    
    try:
        print(f"Downloading image from: {image_url}")
        response = requests.get(image_url)
        response.raise_for_status()
        image_data = response.content

        # Write to a local file
        with open(local_filename, 'wb') as f:
            f.write(image_data)
            
        print(f"File saved locally as: {local_filename}")

        load_dotenv()

        # Run Typhoon OCR on the local file
        print("Running Typhoon OCR...")
        markdown_text = ocr_document(
            pdf_or_image_path=local_filename,   # Works with PDFs or images
            task_type="default",              # Choose between "default" or "structure"
            #Config Base URL and API Key
            api_key=os.getenv('TYPHOON_API_KEY')
        )

        print(f"Saving result to {output_md_filename}...")
        with open(output_md_filename, "w", encoding="utf-8") as md_file:
            md_file.write(markdown_text)
        
        return markdown_text

    except Exception as e:
        return f"Error: {e}"
    finally:
        # Clean up the local file
        if os.path.exists(local_filename):
            print(f"Cleaning up local file: {local_filename}")
            os.remove(local_filename)

In [4]:
# The URL for the main TCAS page
TARGET_URL = 'https://www.mytcas.com/'

# The suffix of images
FILE_SUFFIX1 = 'TGAT-TPAT.jpg'
FILE_SUFFIX2 = 'A-Level.jpg'

# The output file names
FILE_MD1 = 'TGAT-TPAT.md'
FILE_MD2 = 'A-Level.md'


In [5]:
# Get full file_url for TGAT/TPAT image from HTML scraping
file_url1 = scrape_event_links(TARGET_URL, FILE_SUFFIX1)
print(file_url1)

# Extract data from TGAT/TPAT image using TyphoonOCR
ocr_text1 = ocr_image_from_url(file_url1, FILE_MD1)
print(ocr_text1)

https://assets.mytcas.com/69/TCAS69-TGAT-TPAT.jpg
File saved locally as: temp_ocr_image.jpg
Running Typhoon OCR...
Saving result to TGAT-TPAT.md...
Cleaning up local file: temp_ocr_image.jpg
# TGET/TPAT

## ปฏิทินการสมัครสอบและปฏิทินการสอบ

### TPAT1
| ตุลาคม | 1-20 ต.ค. 68 |
| --- | --- |
| สโมสรสอบรายวิชา TPAT1 วิชาเฉพาะ กสพท ผ่านระบบบริษัทสมัครสอบของ กสพท |  |

| กุมภาพันธ์ | 14 ก.พ. 69 |
| --- | --- |
| สอบ TPAT1 วิชาเฉพาะ กสพท ดีตามข่าวสารเพิ่มเติมได้ที่เว็บไซต์ของ กสพท https://cotmesadmission.com |  |

### TGAT/TPAT2-5
| ตุลาคม | 28 ต.ค. 68 เป็นต้นไป |
| --- | --- |
| ลงทะเบียนใช้งานระบบ TCAS69 ที่ student.mytcas.com |  |

| พฤศจิกายน | 30 ต.ค. - 6 พ.ย. 68 |
| --- | --- |
| สโมสรสอบรายวิชา TGAT/TPAT2-5 (สอบด้วยกระดาษ / เครื่องคอมพิวเตอร์) * สอนด้วยเครื่องคอมพิวเตอร์ (CBT) จัดสอบเฉพาะวิชา TGAT และ TPAT3 (สอบเป็นภาษาอังกฤษ) และสมัครสอบแบบ first come, first served * สอนด้วยกระดาษ (PBT) สมัครและเลือกสนามสอบแบบเรียงลำดับ 5 ลำดับ |  |

| ธันวาคม | 29 พ.ย. - 15 ธ.ค. 68 |
| --- | --- |
|

In [6]:
# Get full file_url for A-Level image from HTML scraping
file_url2 = scrape_event_links(TARGET_URL, FILE_SUFFIX2)
print(file_url2)

# Extract data from A-Level image using TyphoonOCR
ocr_text2 = ocr_image_from_url(file_url2, FILE_MD2)
print(ocr_text2)

https://assets.mytcas.com/69/TCAS69-A-Level.jpg
File saved locally as: temp_ocr_image.jpg
Running Typhoon OCR...
Saving result to A-Level.md...
Cleaning up local file: temp_ocr_image.jpg
# A-Level

## ปฏิทินการสมัครสอบและปฏิทินการสอบ

### มกราคม
- **30 ม.ค.-8 ก.พ. 69**
  - สโมสรสอบรายวิชา A-Level (สอบด้วยกระดาษเท่านั้น)
  - โดยเลือกสนามสอบแบบเรียงลำดับ 5 ลำดับ

### กุมภาพันธ์
- **28 ก.พ.-16 มี.ค. 69**
  - พิมพ์บัตรที่นั่งสอบรายวิชา A-Level

### มีนาคม
| วัน | เวลา |
| --- | --- |
| 14 มี.ค. 69 | 08.30 - 10.00 น. |
| 15 มี.ค. 69 | 11.00 - 12.30 น. |
| 16 มี.ค. 69 | 13.30 - 15.00 น. |
| 27-31 มี.ค. 69 | 15.30 - 17.00 น. |

### เมษยน
| วัน | เวลา |
| --- | --- |
| 20 เม.ย. 69 | 08.30 - 10.00 น. |
| 21-27 เม.ย. 69 | 11.00 - 12.30 น. |

### ค่าธรรมเนียม
| ค่าสมัคร A-Level | วิชาละ 100 บาท |
| --- | --- |
| ยื่นคำร้องขอทบทวนผลสอบ A-Level | วิชาละ 100 บาท สูงสุดไม่เกิน 300 บาทต่อคน |

### ปฏิทินการสอบ
#### 14 มี.ค. 69
| วัน | เวลา | รหัสวิชา | ชื่อวิชา |
| --- | --- | --- | --- |
| 14 มี.ค. 6